# In this example we select a list of science window and submit it to ODA to obtain an image

In [ ]:
# with no surprise, we do it for the Crab
source_name='Crab'

## Astroquery can be used to retrieve the list

We build a convenience function.

For the scope of this socumentation, we make sure the `astroquery` library version, is within a specific version range. For newer (or older) version, please consult the offical documentation.

In [ ]:
#Example of functions to select a science window list
import time

import astroquery.heasarc
from astropy.coordinates import SkyCoord
from astropy import units as u

Heasarc = astroquery.heasarc.Heasarc()


def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            mission = 'intscw',
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )
    R.sort('SCW_ID')

    return R['SCW_ID'], R['SCW_VER']


assert astroquery.__version__ >= '0.4.2.dev6611' and astroquery.__version__ <= '0.4.7'

# it means it's our fork
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In [ ]:
from astroquery.simbad import Simbad
from astropy import coordinates as coord

import json

import os
import shutil
import random

from astropy.io import fits

from scipy import stats
result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))
ra=source_coord.ra.deg
dec=source_coord.dec.deg
radius=8.

- We get science windows within 10 degrees from the Crab position 
- between the given dates expressed in the ISOT format

In [ ]:
scwlist,version = get_scw_list(ra,dec, 10, '2020-01-01T00:00:00', '2020-06-01T00:00:00')

- We pick at random nscw = 10 science window from the list (this is as ODA backend does)

In [ ]:

nscw = 10
if len(scwlist) == 0:
    raise Exception('No science windows')

scw_pick = [(s+"."+v).strip() for s,v in zip(scwlist,version)]

if nscw >0 and len(scw_pick)>=nscw:
    random.seed(0)
    scw_pick = random.sample(scw_pick, nscw)
scw_pick = sorted(scw_pick)

In [ ]:
scw_pick

- Token for a given user 

In [ ]:
token=''

In [ ]:
#You can provide a valid token as explained in the 'Authentication' example or skip this cell
import getpass
token = getpass.getpass('Insert the token')

## Let's get some logging

This is to help visualizing the progress.

- WANRING is the default level
- INFO writes some more information
- DEBUG is maily for developers and issue tracking


In [ ]:
import logging
#default
#logging.getLogger().setLevel(logging.WARNING)
#slightly more verbose
logging.getLogger().setLevel(logging.INFO)
#all messages
#logging.getLogger().setLevel(logging.DEBUG)

logging.getLogger('oda_api').addHandler(logging.StreamHandler()) 

In [ ]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

par_dict = {
"E1_keV": "28",
"E2_keV": "40",
"detection_threshold": "7",
"instrument": "isgri",
"osa_version": "OSA11.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
"integral_data_rights": "all-private"    
}

if token != '':
    par_dict.update({'token': token})

data_collection = disp.get_product(**par_dict)

In [ ]:
data_collection.mosaic_image_0_mosaic.show()

In [ ]:
data_collection.mosaic_image_0_mosaic.show_meta()

In [ ]:
from oda_api.plot_tools import OdaImage
#interactive
%matplotlib inline

im=OdaImage(data_collection.mosaic_image_0_mosaic)
im.show(unit_ID=4)